## Libraries

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

## Load Files

In [2]:
train=pd.read_csv('train.csv')
train.head()


,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [3]:
test=pd.read_csv('test.csv')
test.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


## Add stopwords

In [4]:
from nltk.corpus import stopwords
import string

In [5]:
mylist=list((set(train['airline'])))
airline=set()
for i in mylist:
    airline.add(i.lower())
airline

{'american', 'delta', 'southwest', 'united', 'us airways', 'virgin america'}

In [6]:
stop=stopwords.words('english')
df=pd.DataFrame(stop)
df.to_csv('stop_words.csv',index=False)

In [7]:
def add_stopwords(training_data,airline):
    for i in training_data:
        for j in i.split(' '):
            k=j;
            #print(k)
            if (len(k)==0):
                continue
            if (k[0]=='@'):
                airline.add(j)
                airline.add(j[1:])
            if (k.isnumeric()==True):
                airline.add(k)
            if (k[0]>='0' and k[0]<='9'):
                airline.add(k)
    airline
add_stopwords(train['text'],airline)

In [8]:
puncts=list(string.punctuation)
stop+=puncts
stop+=list(airline)

## Training Data,Testing Data and Target

In [9]:
def setdata(data):
    training_data=[]
    for i in data:
        words=""
        for j in i.split(' '):
            if len(j)==0:
                continue
            if '0'<=j[0]<='9':
                continue
            if j[0]=='@':
                continue
            if len(j)>=4:
                if (j[:4]=='http'):
                    continue
            if not j.isnumeric():
                words=words+j+' '
        training_data.append(words)
    return training_data

In [10]:
training_data=setdata(train['text'])
training_data

['I am scheduled for the morning, days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled ',
 'seeing your workers time in and time out going above and beyond is why I love flying with you guys. Thank you! ',
 'Flew ORD to Miami and back and had great crew, service on both legs. THANKS ',
 "that's horse radish 😤🐴 ",
 'so our flight into ORD was delayed because of Air Force One, but the last flight to SBN is at mins from now we just landed. ',
 'Why did you load us in this flying sardine can if you knew the pilots were hours Late Flight?? #incompetent beyond belief ',
 "that is a stock response. Delays not as frustrating as poor cust serv &amp; being told by ppl to wait &amp; they'd come back but did not. ",
 "That'd be nice! Hoping to rack up enough miles to take a trip to Seattle and enjoy a perfect latte in the city of coffee. ",
 'frankly worse customer service ever. Problems will happen, how you deal defines a company. Never again United. ',
 "

In [11]:
Y=train['airline_sentiment']
Y

0        negative
1        positive
2        positive
3        negative
4        negative
           ...   
10975     neutral
10976    positive
10977    negative
10978    negative
10979    negative
Name: airline_sentiment, Length: 10980, dtype: object

In [12]:
testing_data=setdata(test['text'])
testing_data

["In car gng to DFW. Pulled over ago - very icy roads. On-hold with AA since Can't reach arpt for AA2450. Wat do? ",
 'after all, the plane didn’t land in identical or worse) conditions at GRK according to METARs. ',
 "can't believe how many paying customers you left high and dry with no reason for flight Cancelled Flightlations Monday out of BDL! Wow. ",
 'I can legitimately say that I would have rather driven cross country than flown on US Airways. ',
 'still no response from AA. great job guys! ',
 'we have developers flying down tmrw morn. w/45 min layover, there is an earlier flight to have layover, can move them up? ',
 'hello??? Anyone there? ',
 'Mr. Husain u shld protest as well when one of ur party member Rehman Malik delayed a PIA flight for hours..??? ',
 "not likely, flightaware says plane is still in Durango and hasn't departed. ",
 "they don't even give an option to hold.. Just say lines are busy Plz try Late Flightr ",
 'your announcement for pre boarding only addresses

## Data Cleaning

In [13]:
training_data=[[j for j in word_tokenize(i) if not (j in stop)] for i in training_data]
testing_data=[[j for j in word_tokenize(i) if not (j in stop)] for i in testing_data]

In [14]:
training_data[0]

['I',
 'scheduled',
 'morning',
 'days',
 'fact',
 'yes..not',
 'sure',
 'evening',
 'flight',
 'one',
 'Cancelled',
 'Flightled']

In [15]:
testing_data[0]

['In',
 'car',
 'gng',
 'DFW',
 'Pulled',
 'ago',
 'icy',
 'roads',
 'On-hold',
 'AA',
 'since',
 'Ca',
 "n't",
 'reach',
 'arpt',
 'AA2450',
 'Wat']

In [16]:
np.asarray(training_data).shape,np.asarray(testing_data).shape


((10980,), (3660,))

In [17]:
training_data[0][0]

'I'

## Lemmatizing

In [18]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [19]:
lemm=WordNetLemmatizer()

In [20]:
def get_simple(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [21]:
def lemmatizing(training_data):
    tweets=[]
    for i in training_data:
        words=[]
        for j in i:
            pos=pos_tag([j.lower()])
            word=lemm.lemmatize(j.lower(),pos=get_simple(pos[0][1]))
            words.append(word)
        tweets.append(words)
    return tweets

In [22]:
training_data=lemmatizing(training_data)
testing_data=lemmatizing(testing_data)

In [23]:
np.asarray(training_data).shape,np.asarray(testing_data).shape


((10980,), (3660,))

## Making Feature Set

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [25]:
def join_tweets(tweets):
    join_tweet=[]
    for i in tweets:
        sentence=""
        for j in i:
            sentence=sentence+" "+j
        join_tweet.append(sentence.strip())
    return join_tweet

In [26]:
training_data=join_tweets(training_data)
testing_data=join_tweets(testing_data)

In [27]:
temp_train=training_data
temp_test=testing_data

In [28]:
count_vec=CountVectorizer(max_features=10000,ngram_range=(1,2),binary=False) # count vctorizer finds most frquent features
training_data=count_vec.fit_transform(training_data)

In [29]:
count_vec.get_feature_names()

['00',
 '1000',
 '130',
 '15',
 '25',
 '350',
 '3fq3xelbon',
 '400',
 '42',
 '42 be',
 '45',
 '50',
 '700',
 '700 flight',
 '75',
 '85832',
 'a1',
 'a320',
 'aa',
 'aa amp',
 'aa employee',
 'aa flight',
 'aa help',
 'aa2444',
 'aadvantage',
 'abandon',
 'abc',
 'ability',
 'able',
 'able check',
 'able get',
 'able help',
 'able leave',
 'able put',
 'able reach',
 'able use',
 'aboard',
 'about',
 'abq',
 'absolute',
 'absolute bad',
 'absolutely',
 'absolutely disgust',
 'absolutely the',
 'absurd',
 'abt',
 'abt flight',
 'abuse',
 'abysmal',
 'ac',
 'accept',
 'accept call',
 'acceptable',
 'accepted',
 'access',
 'access flight',
 'access journal',
 'access line',
 'access website',
 'accident',
 'accident way',
 'accidentally',
 'accommodate',
 'accommodation',
 'accomplish',
 'accord',
 'accordingly',
 'account',
 'account confirm',
 'account help',
 'account see',
 'account they',
 'accountability',
 'acct',
 'accts',
 'accurate',
 'accuse',
 'acknowledge',
 'acknowledgment',


In [30]:
#count_vec=CountVectorizer(max_features=5000) # count vctorizer finds most frquent features
testing_data=count_vec.transform(testing_data)
#training_data[0]

In [31]:
count_vec.get_feature_names()

['00',
 '1000',
 '130',
 '15',
 '25',
 '350',
 '3fq3xelbon',
 '400',
 '42',
 '42 be',
 '45',
 '50',
 '700',
 '700 flight',
 '75',
 '85832',
 'a1',
 'a320',
 'aa',
 'aa amp',
 'aa employee',
 'aa flight',
 'aa help',
 'aa2444',
 'aadvantage',
 'abandon',
 'abc',
 'ability',
 'able',
 'able check',
 'able get',
 'able help',
 'able leave',
 'able put',
 'able reach',
 'able use',
 'aboard',
 'about',
 'abq',
 'absolute',
 'absolute bad',
 'absolutely',
 'absolutely disgust',
 'absolutely the',
 'absurd',
 'abt',
 'abt flight',
 'abuse',
 'abysmal',
 'ac',
 'accept',
 'accept call',
 'acceptable',
 'accepted',
 'access',
 'access flight',
 'access journal',
 'access line',
 'access website',
 'accident',
 'accident way',
 'accidentally',
 'accommodate',
 'accommodation',
 'accomplish',
 'accord',
 'accordingly',
 'account',
 'account confirm',
 'account help',
 'account see',
 'account they',
 'accountability',
 'acct',
 'accts',
 'accurate',
 'accuse',
 'acknowledge',
 'acknowledgment',


In [32]:
from sklearn.ensemble import RandomForestClassifier

In [49]:
clf=RandomForestClassifier(n_estimators=200,random_state=0)

In [50]:
clf.fit(training_data,Y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [35]:
#clf.score(x_test_features,y_test)

In [51]:
y_pred=clf.predict(testing_data)

In [52]:
y_pred

array(['negative', 'negative', 'negative', ..., 'negative', 'positive',
       'negative'], dtype=object)

In [53]:
np.savetxt('random_forest_prediction.csv',y_pred,fmt='%s')

In [39]:
#training_data=temp_train
#testing_data=temp_test

## Using Multinomial NB


In [40]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [41]:
clf=OneVsRestClassifier(SVC(random_state=0,gamma=100))

In [42]:
clf.fit(training_data,Y)

OneVsRestClassifier(estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=0.0,
                                  decision_function_shape='ovr', degree=3,
                                  gamma=100, kernel='rbf', max_iter=-1,
                                  probability=False, random_state=0,
                                  shrinking=True, tol=0.001, verbose=False),
                    n_jobs=None)

In [43]:
ovsr_output=clf.predict(testing_data)


In [44]:
np.savetxt('one_vs_rest_prediction.csv',ovsr_output,fmt='%s')

In [45]:
clf=OneVsRestClassifier(RandomForestClassifier(random_state=0))

In [46]:
clf.fit(training_data,Y)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                            

In [47]:
ovsr_output=clf.predict(testing_data)

In [48]:
np.savetxt('one_vs_rest_prediction.csv',ovsr_output,fmt='%s')